In [23]:
import pandas as pd

# Creazione del DataFrame con le proprietà delle molecole
data = {
    'Molecola': ['M1', 'M2', 'M3', 'M4', 'M5'],
    'Temp_Ebollizione (°C)': [350, 400, 310, 500, 400],
    'pH': [6, 10, 2, 7, 9],
    'Ionizzabile': [True, True, False, True, True]
}

df = pd.DataFrame(data)

# Funzione per determinare il tipo di cromatografia appropriato per il miscuglio
def determina_cromatografia(df):
    # Controlla se tutte le temperature di ebollizione sono maggiori di 300°C
    if df['Temp_Ebollizione (°C)'].min() >= 300:
        tipo_cromatografia = 'GC'
    else:
        # Controlla se il pH è compreso tra 3 e 11
        if df['pH'].between(3, 11).all():
            # Controlla se tutte le molecole nel miscuglio sono ionizzabili
            if df['Ionizzabile'].all():
                tipo_cromatografia = 'IC'
            else:
                tipo_cromatografia = 'HPLC'
        else:
            tipo_cromatografia = 'HPLC'
    
    # Aggiunta per gestire il caso in cui tutte le molecole sono ionizzabili ma la temp. di ebollizione è < 300°C
    if df['Ionizzabile'].all() and df['Temp_Ebollizione (°C)'].min() < 300:
        tipo_cromatografia = 'IC'
    
    return tipo_cromatografia

# Determina il tipo di cromatografia appropriato per il miscuglio
tipo_cromatografia_miscuglio = determina_cromatografia(df)

print(f"Tipo di cromatografia consigliato per il miscuglio: {tipo_cromatografia_miscuglio}")


Tipo di cromatografia consigliato per il miscuglio: GC
